In [1]:
import json
import time
import re
import os
import glob

from urllib.parse import urljoin
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [2]:
lst_uls = [
    'https://www.wayfair.com/decor-pillows/pdp/gracie-oaks-kaehi-brushed-nickel-mirror-wall-mirror-w008667613.html',  
    
    'https://www.wayfair.com/furniture/pdp/ebern-designs-corner-tv-stand-for-50-tv-modern-living-room-with-storage-space-low-profile-tv-cabinet-w010361605.html',  
    
    'https://www.wayfair.com/furniture/pdp/latitude-run-garysburg-solid-wood-5-drawer-accent-chest-w007680564.html'
    
]
lst_uls

['https://www.wayfair.com/decor-pillows/pdp/gracie-oaks-kaehi-brushed-nickel-mirror-wall-mirror-w008667613.html',
 'https://www.wayfair.com/furniture/pdp/ebern-designs-corner-tv-stand-for-50-tv-modern-living-room-with-storage-space-low-profile-tv-cabinet-w010361605.html',
 'https://www.wayfair.com/furniture/pdp/latitude-run-garysburg-solid-wood-5-drawer-accent-chest-w007680564.html']

In [2]:
lst_urls = [
    'https://www.wayfair.com/outdoor/pdp/ebern-designs-planter-on-stand-ceramic-planter-on-wooden-stand-contemporary-ivorybeige-gradient-design-indoor-or-outdoor-plant-stand-decor-w004578330.html?piid=985442586',
    'https://www.wayfair.com/outdoor/pdp/dakota-fields-ceramic-planter-on-bamboo-wood-stand-plant-stand-with-contemporary-dotted-design-indoor-or-outdoor-plants-and-succulents-w003023309.html?piid=1195818433%2C1195776358',
    'https://www.wayfair.com/outdoor/pdp/wrought-studio-elayna-2-piece-ceramic-pot-planter-set-w005313805.html?piid=228007296'
]
lst_urls

['https://www.wayfair.com/outdoor/pdp/ebern-designs-planter-on-stand-ceramic-planter-on-wooden-stand-contemporary-ivorybeige-gradient-design-indoor-or-outdoor-plant-stand-decor-w004578330.html?piid=985442586',
 'https://www.wayfair.com/outdoor/pdp/dakota-fields-ceramic-planter-on-bamboo-wood-stand-plant-stand-with-contemporary-dotted-design-indoor-or-outdoor-plants-and-succulents-w003023309.html?piid=1195818433%2C1195776358',
 'https://www.wayfair.com/outdoor/pdp/wrought-studio-elayna-2-piece-ceramic-pot-planter-set-w005313805.html?piid=228007296']

In [ ]:
options = Options()
# options.add_argument("--incognito")
options.add_argument("--disable-geolocation")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")

base_url = 'https://www.wayfair.com/outdoor/pdp/ebern-designs-planter-on-stand-ceramic-planter-on-wooden-stand-contemporary-ivorybeige-gradient-design-indoor-or-outdoor-plant-stand-decor-w004578330.html?piid=985442586'

header =  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
options.add_argument(f'--user-agent={header}')


normal_driver = webdriver.Chrome(options=options)

solver = RecaptchaSolver(driver=normal_driver)
# driver = webdriver.Chrome()
driver.maximize_window()
normal_driver.get(url=base_url)

In [ ]:
captacha_box = driver.find_element(By.CLASS_NAME,"px-captcha-container")
captacha_box

In [ ]:
captacha_box.text

<a class="rc-audiochallenge-tdownload-link">Audio MP3 LInk</a>

In [ ]:
driver.close()
driver.quit()

In [3]:
from selenium_recaptcha_solver import RecaptchaSolver
base_url=lst_uls[2]
base_url

'https://www.wayfair.com/furniture/pdp/latitude-run-garysburg-solid-wood-5-drawer-accent-chest-w007680564.html'

In [6]:
def not_medding_kid(base_url):
    test_ua = 'Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0 Safari/537.36'
    options = Options()
    
    # options.add_argument("--headless")  # Remove this if you want to see the browser (Headless makes the chromedriver not have a GUI)
    options.add_argument("--window-size=1920,1080")
    options.add_argument(f'--user-agent={test_ua}')
    options.add_argument('--no-sandbox')
    options.add_argument("--disable-extensions")
    
    test_driver = webdriver.Chrome(options=options)
    
    solver = RecaptchaSolver(driver=test_driver)
    time.sleep(7)
    test_driver.get(base_url)
    recaptcha_iframe = None
    try:
        time.sleep(2)
        recaptcha_iframe = test_driver.find_element(By.XPATH, '//iframe[@title="reCAPTCHA"]')
    except Exception as e:
        str(e)
    
    
    print('here')
    if recaptcha_iframe:
        try:
            solver.click_recaptcha_v2(iframe=recaptcha_iframe)
            print('captacha found!')
        except RecaptchaException as rre:
            print('captacha not able to resolve')
        except Exception as e:
            print('error came please look into it')
    else:
        print('nah!!')
    
    wait =  WebDriverWait(test_driver, 20)
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'SFPrice')))
    
    time.sleep(5)
    soup = BeautifulSoup(test_driver.page_source,'html.parser')
    
    r1=None; r2=None; r3=None
    try:
        r1 = soup.find('span', class_='ProductRatingNumberWithCount-rating')
        r2 = soup.find('span', class_='ProductRatingNumberWithCount-count').text.strip()
        r3 = soup.find('div',class_='SFPrice').text
    except Exception as e:
        print(type(soup))
        print('not able to get detail')
    
    print(r1,r2,r3)
    time.sleep(3)
    test_driver.close()
    test_driver.quit()

In [7]:
not_medding_kid(base_url)

here
nah!!
<span class="ProductRatingNumberWithCount-rating">4.0</span> 82 Reviews $119.99$155.9923% Off


In [10]:
for i in lst_uls:
    not_medding_kid(i)

here
nah!!
<span class="ProductRatingNumberWithCount-rating">0.0</span> 0 Reviews $163.99
here
nah!!
<span class="ProductRatingNumberWithCount-rating">4.7</span> 7 Reviews $114.99
here
nah!!
<span class="ProductRatingNumberWithCount-rating">4.0</span> 82 Reviews $119.99$155.9923% Off


In [ ]:
test_driver.close()
test_driver.quit()